# Introduction

In the following, we will consider the simplest problem that can be encountered in machine learning. We have an unknown function $x\rightarrow f(x) \in \mathbb{R}$ and a set of points $\mathcal{S}=\{f\left(x_i\right), x_i \in \mathbb{R}\}$ sampled from our function $f$. The goal is to find an approximate function of $f$ that is satisfactory for the real-world problem at hand. We will see in the following how neural network, with the help of deep learning, can handle these problems. The learning process where the correct answer $f(x_i)$ for each input $x_i$ is known is called supervised learning, as it is similar to a teacher knowing the answer and guiding the student to it.

# 1) Single neuron

## Theory

We begin our exploration of neural networks with their fundamental unit : a simple neuron with one input and one output. To the neuron is associated a weight $W$ and it is feeded with an input $x$ coming from the samples $\mathcal{S}$. Based on this, the neuron outputs

\begin{equation*}
    y = Wx.
\end{equation*}

In supervised learning, the right answer $\xi$ is known. Hence, we can quantify the difference between $\xi$ and the prediction $y$ of the neuron

\begin{equation*}
    \delta = y-\xi = Wx-\xi
\end{equation*}

A mean to quantify the error of the neuron is the squared deviation

\begin{equation*}
    \epsilon = \left(y-\xi \right)^2 = \left(W x-\xi \right)^2
\end{equation*}

The goal in the learning process is to minimize $\epsilon$ with respect to the weight $W$. For this, gradient descent is used, where we take the slope of the function $W\rightarrow\epsilon(W)$ to push the weight $W$ in the direction of the minimum of the function. Note that in the case of the squared deviation, the slope is given by

\begin{align}
    \frac{\partial \epsilon}{\partial W} &= \frac{\partial}{\partial W} \left( Wx - \xi\right)^2 \\
                                        &= 2\left( Wx - \xi\right)x \\
                                        & \propto \delta x \\     
\end{align}

If the slope is too steep, we might push the weight too far and go beyond the minimum. To avoid this, we define a learning rate $\alpha$ which is a given parameter of the learning process. Finally, we update the weight with the following formula

\begin{equation}
    W' = W-\alpha\delta x
\end{equation}

Now, let's look at how to code it in Julia.

## Julia code

We define a single neuron as a new structure that encodes a weight, $W$. This weight must be updated, so the structure has to be mutable.

In [17]:
mutable struct singleNeuronModel
    W::Float64
end

We define on this structure a function which gives the prediction of the neuron as a product of the weight with the input.

In [18]:
(m::singleNeuronModel)(x) = m.W * x

Let's instantiate a single neuron with a random weight.

In [19]:
model = singleNeuronModel(rand())

singleNeuronModel(0.9234242035653861)

Let's test this neuron with some dummy data and compute the error.

In [20]:
x = 5
ξ = 2
δ = model(x)- ξ
ϵ = δ^2
println(model(x))
println(δ)
println(ϵ)

4.61712101782693
2.61712101782693
6.8493224219514675


Of course, as the weight is random, so is the initial error. We now want to train our neuron to reduce the error. To do this, we update the weight recursively with the help of gradient descent. In the train! function, we denote the input $x\equiv x_\mathrm{train}$ and the right answer $\xi\equiv y_\mathrm{train}$ as these are the training data on which the neuron can learn.

In [21]:
function train!(model, xtrain, ytrain, α, iteration)
    for i in 1:iteration
        y = model(xtrain) # Compute the prediction from the model
        δ = y - ytrain # Difference between the prediction and the expectation
        ϵ = δ^2 # Error computed with the mean squared
        model.W = model.W -(δ*xtrain*α) # Update the weight
        @show ϵ
    end
end

train! (generic function with 1 method)

This function takes as arguments our neuron, an input and the corresponding answer but also a learning rate $\alpha$ and the number of times that we want the weight to be updated. Let's see how the error evolves at each iteration.

In [25]:
train!(model, x, ξ, 0.01, 30)

ϵ = 0.00025613844039918266
ϵ = 0.00014407787272453492
ϵ = 8.104380340755488e-5
ϵ = 4.558713941675112e-5
ϵ = 2.5642765921919132e-5
ϵ = 1.4424055831082886e-5
ϵ = 8.113531404981593e-6
ϵ = 4.5638614153030954e-6
ϵ = 2.567172046107991e-6
ϵ = 1.4440342759357448e-6
ϵ = 8.122692802142568e-7
ϵ = 4.569014701205194e-7
ϵ = 2.5700707694279217e-7
ϵ = 1.4456648078023616e-7
ϵ = 8.131864543888285e-8
ϵ = 4.57417380593716e-8
ϵ = 2.5729727658396525e-8
ϵ = 1.4472971807874759e-8
ϵ = 8.141046641889483e-9
ϵ = 4.579338736062834e-9
ϵ = 2.575878039035344e-9
ϵ = 1.4489313969658332e-9
ϵ = 8.150239107869421e-10
ϵ = 4.584509498176549e-10
ϵ = 2.578786592759966e-10
ϵ = 1.4505674584007381e-10
ϵ = 8.159441953704723e-11
ϵ = 4.589686099109336e-11
ϵ = 2.581698430410536e-11
ϵ = 1.452205367190543e-11


The error reduces as expected ! Of course, a single neuron is unable to approximate complex functions. To get more flexibility, we need more neurons and some sort of non-linearity.

# 2) Single layer linear network

## Theory

A neural network can have several layers of neurons, this is called the depth of the network. The number of layers is noted $D$ and for this section we take $D=1$. The first thing we can generalize about a single-layer network is the number of inputs $N_0$. As there are $N_0$ inputs, there is $N_0$ weights. To simplify the notation of these inputs and weights, we can put them in the form of vectors

\begin{equation}
    \mathbf{W}^\top = \left( \begin{array}{ccc}
        W_1 \\
        W_2 \\
        \vdots \\
        W_{N_0} \end{array} \right)\qquad \qquad \qquad
     \mathbf{x} = \left( \begin{array}{ccc}
        x_1 \\
        x_2 \\
        \vdots \\
        x_{N_0} \end{array} \right).
\end{equation}

The output of the single neuron is

\begin{equation}
    y = \mathbf{W} \mathbf{x} = W_1 x_1 + W_2 x_2 + ... + W_{N_0} x_{N_0}.
\end{equation}

The number of neurons on the last layer determines the number of outputs. In this case, there is only one layer with one neuron and therefore only one output. To increase the number of outputs, we increase the number of neurons in our layer. We denote this number $N_1$ and refer to it as the width of the layer. The $N_0$ inputs are thus each linked to $N_1$ neurons and our different weights will be organised in the form of a matrix.

\begin{equation}
    \mathbf{W} = \left( \begin{array}{ccc}
        W_{11} & W_{12} & \cdots & W_{1 N_0} \\
        W_{21} & W_{22} & \cdots & W_{2 N_0} \\
        \vdots & \ddots & \ddots & \vdots\\
        W_{N_1 1} & W_{N_1 2} & \cdots & W_{N_1 N_0}\end{array} \right)\qquad \qquad \qquad
     \mathbf{x} = \left( \begin{array}{ccc}
        x_1 \\
        x_2 \\
        \vdots \\
        x_{N_0} \end{array} \right).
\end{equation}

Hence, the $i$-th output of the network is

\begin{equation}
    y_i = (\mathbf{W} \mathbf{x})_i = W_{i1} x_1 + W_{i2} x_2 + \cdots + W_{iN_0} x_{N_0}.
\end{equation}

To go further, we can also add a bias to each prediction, which we organise in the form of a vector $\mathbf{b}^\top = (b_1, b_2, \cdots, b_{N_1})$. The $i$-th prediction is therefore

\begin{equation}
    y_i = (\mathbf{W} \mathbf{x} + \mathbf{b})_i = W_{i1} x_1 + W_{2i} x_2 + \cdots + W_{iN_0} x_{N_0} + b_i.
\end{equation}

To give a few examples, when we want a network that takes a set of data and outputs a value, we'll simply take $N_1$ = 1. When we have an object (an image, a wave function, etc.) that we want to classify into different categories, then $N_1$ will be the number of categories and we will then apply a normalisation function at the end of the network to ascertain the probabilities that an object belongs to a specific category.

To each prediction $y_i$, we can associate a difference between it and the expected value $\xi_i$ and compute the squared error

\begin{equation}
    \delta_i = y_i - \xi_i
\end{equation}

\begin{equation}
    \epsilon_i = (y_i - \xi_i)^2,
\end{equation}

the total error being simply the sum of the errors

\begin{equation}
    \epsilon = \sum_{i=1}^{N_1}(y_i - \xi_i)^2.
\end{equation}

To train the network, we proceed in the same way as for the single neuron case. The derivative of the error associated with a weight is used to update the weight. This gives us

\begin{aligned}
        \frac{\partial \epsilon_i}{\partial W_{ij}} &= \frac{\partial}{\partial W_{ij}} \left( W_{i1} x_1 + W_{i2} x_2 + \cdots + W_{iN_0} x_{N_0} + b_i - \xi_i\right)^2 \\
        &= 2\left( W_{i1} x_1 + W_{i2} x_2 + \cdots + W_{iN_0} x_{N_0} + b_i - \xi_i\right)x_j \\
        &\propto x_j \delta_i
\end{aligned}

And the updated weight is

\begin{equation}
    W_{ij}' = W_{ij} - \alpha x_j \delta_i.
\end{equation}

By performing the same derivative for the bias, we can see that it is updated using the following formula

\begin{equation}
    b_i' = b_i - \alpha \delta_i.
\end{equation}

## Julia code

Once again, we define the shape of our network using a structure and use it to define the prediction function based on the inputs.

In [29]:
mutable struct LinearOneLayerModel
    W::Array{Float64,2}
    b::Vector{Float64}
end

(m::LinearOneLayerModel)(x) = m.W * x + m.b

We specify the number of inputs and outputs and then instantiate the model with random weights. The transposed matrix of the weights is taken directly so that the transposition does not have to be performed each time.

In [30]:
nInput = 3;
nOutput = 7;

model = LinearOneLayerModel(rand(nOutput,nInput), rand(nOutput));

To access an element of the structure, use the following syntax. We can see that we have an Array and a Vector.

In [31]:
println(model.W)
println(model.b)

[0.1520034834835423 0.957429029919127 0.14813348601169585; 0.9197675283814533 0.5142917560408474 0.8350692804473666; 0.1674918425016062 0.6899595716525422 0.4666664118719377; 0.5233003333324129 0.5771253393237994 0.30390153657312047; 0.8612531911678054 0.7217587533906575 0.9737972753431126; 0.9602265210095754 0.3767848423998237 0.6478058661522801; 0.9355628852936813 0.3447602646319241 0.6959130467314106]
[0.9205607543935563, 0.33942461922785727, 0.1371652149165482, 0.6763682352642969, 0.8557160285505618, 0.7719340533560743, 0.6734687936050778]


We can then give ourselves data and test the network.

In [37]:
x = rand(nInput)
ξ = rand(nOutput)

y = model(x)
δ = y - ξ
ϵ = δ.^2
ϵTot = sum(ϵ)
println(δ)
println(ϵ)
println(ϵTot)

[-0.0298739149490298, -0.06687510423982779, -0.8132134133770582, -0.30759466427290494, 0.04739860818298969, 0.26581121440893485, 0.4391862227989465]
[0.0008924507943818662, 0.004472279567087832, 0.6613160556963662, 0.0946144774891611, 0.002246628057684577, 0.07065560170555274, 0.19288453829640587]
1.0270820316066402


To improve the result, we define a function that will train the network. For the record, when going through a matrix in Julia, it's quicker to do it column by column (see [performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/)).

In [33]:
function train!(model, x, ξ, iteration, α)
    for χ in 1:iteration
        y = model(x) # Compute the prediction from the model
        δ = y - ξ # Difference between the prediction and the expectation

        # Error computed with the mean squared
        ϵ = δ.^2

        # Update the weights
        for j in 1:length(x)
            for i in 1: length(ξ)
                model.W[i,j] = model.W[i,j] -(α*x[j]*δ[i])
                model.b[i] = model.b[i] - α*δ[i]
            end
        end
        @show sum(ϵ)
    end
end

train! (generic function with 1 method)

In [38]:
train!(model, x, ξ, 30, 0.1)

sum(ϵ) = 1.0270820316066402
sum(ϵ) = 0.333075885536838
sum(ϵ) = 0.10801429887017776
sum(ϵ) = 0.03502832017277846
sum(ϵ) = 0.0113594517296399
sum(ϵ) = 0.003683794796939677
sum(ϵ) = 0.0011946302012579622
sum(ϵ) = 0.00038741064484461757
sum(ϵ) = 0.00012563470066375088
sum(ϵ) = 4.0742499518053464e-5
sum(ϵ) = 1.3212522163134891e-5
sum(ϵ) = 4.284733238664879e-6
sum(ϵ) = 1.3895105491478179e-6
sum(ϵ) = 4.506090481363713e-7
sum(ϵ) = 1.4612952336847586e-7
sum(ϵ) = 4.7388834485689395e-8
sum(ϵ) = 1.536788447775668e-8
sum(ϵ) = 4.983702930948641e-9
sum(ϵ) = 1.6161817808985615e-9
sum(ϵ) = 5.241170240537911e-10
sum(ϵ) = 1.6996767204709463e-10
sum(ϵ) = 5.5119387874142966e-11
sum(ϵ) = 1.7874851629236467e-11
sum(ϵ) = 5.796695738463627e-12
sum(ϵ) = 1.8798299520860837e-12
sum(ϵ) = 6.096163756809324e-13
sum(ϵ) = 1.9769454450427233e-13
sum(ϵ) = 6.411102854417033e-14
sum(ϵ) = 2.0790781017661814e-14
sum(ϵ) = 6.742312273590084e-15
